In this guide, we'll see how to add ad-hoc tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

Ref: https://python.langchain.com/v0.2/docs/how_to/tools_prompting/

In [24]:
from IPython.display import display, Markdown, JSON

In [1]:
from langchain_community.llms import Ollama
model = Ollama(model='phi3')

### Create a tool

In [4]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [44]:
multiply.dict()

{'name': 'multiply',
 'description': 'Multiply two numbers together.',
 'args_schema': pydantic.v1.main.multiplySchema,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'func': <function __main__.multiply(x: float, y: float) -> float>,
 'coroutine': None}

In [11]:
# invoke a tool
multiply.invoke({'x':4, 'y':12})

48.0

### Render tools description

In [12]:
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.


### Create a prompt

In [16]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""
print(system_prompt)

You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.



In [26]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)
prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are an assistant that has access to the following set of tools. \nHere are the names and descriptions for each tool:\n\nmultiply(x: float, y: float) -> float - Multiply two numbers together.\nadd(x: int, y: int) -> int - Add two numbers.\n\nGiven the user input, return the name and input of the tool to use. \nReturn your response as a JSON blob with 'name' and 'arguments' keys.\n\nThe `arguments` should be a dictionary, with keys corresponding \nto the argument names and the values corresponding to the requested values.\n")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [28]:
from langchain_core.runnables import RunnablePassthrough

chain = {'input': RunnablePassthrough()} | prompt | model
message = chain.invoke('what is 3 plus 41')

In [31]:
# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

 {
  "name": "add",
  "arguments": {
    "x": 3,
    "y": 41
  }
}


### Add output parser

In [35]:
from langchain_core.output_parsers import JsonOutputParser

chain = {'input': RunnablePassthrough()} | prompt | model | JsonOutputParser()
message = chain.invoke("what's thirteen times 4")

print(type(message))
print(message)

<class 'dict'>
{'name': 'multiply', 'arguments': {'x': 13, 'y': 4}}


### Invoking the tool

In [39]:
def invoke_tool(tool_definition, config=None):
    name = tool_definition['name']
    arguments = tool_definition['arguments']

    # create a dictionary from list to find the tool by name
    tools_as_dict = {tool.name: tool for tool in tools}
    tool = tools_as_dict[name]
    return tool.invoke(arguments, config=config)

In [40]:
# lets try invoke_tool function

invoke_tool({'name': 'multiply', 'arguments': {'x': 13, 'y': 4}})

52.0

### Put everything together

In [47]:
chain = {'input': RunnablePassthrough()} | prompt | model | JsonOutputParser() | invoke_tool
message = chain.invoke("what's thirteen times 4")
print(message)

52.0


### Return tool data and result together

In [52]:
chain = {'input': RunnablePassthrough()} | prompt | model | JsonOutputParser() | RunnablePassthrough.assign(my_output=invoke_tool)
message = chain.invoke("what's thirteen times 4")
print(message)

{'name': 'multiply', 'arguments': {'x': 13.0, 'y': 4.0}, 'my_output': 52.0}
